In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [98]:
data = pd.read_csv("Debernardi et al 2020 data.csv")

In [99]:
data.drop(columns=["sample_id","patient_cohort","sample_origin","stage","benign_sample_diagnosis"],axis=1,inplace=True)

In [100]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Apply Label Encoding to each categorical column
for column in ["sex","diagnosis","age"]:
    data[column] = label_encoder.fit_transform(data[column])

In [101]:
data

,age,sex,diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,6,0,0,11.7,1.83222,0.893219,52.948840,654.282174,1262.000
1,54,0,0,NaN,0.97266,2.037585,94.467030,209.488250,228.407
2,24,1,0,7.0,0.78039,0.145589,102.366000,461.141000,NaN
3,34,1,0,8.0,0.70122,0.002805,60.579000,142.950000,NaN
4,35,1,0,9.0,0.21489,0.000860,65.540000,41.088000,NaN
...,...,...,...,...,...,...,...,...,...
585,41,1,2,NaN,0.52026,7.058209,156.241000,525.178000,NaN
586,44,0,2,NaN,0.85956,8.341207,16.915000,245.947000,NaN
587,36,1,2,NaN,1.36851,7.674707,289.701000,537.286000,NaN
588,48,0,2,NaN,1.33458,8.206777,205.930000,722.523000,NaN


In [102]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data[['plasma_CA19_9', 'creatinine', 'LYVE1','REG1B', 'TFF1', 'REG1A']] = scaler.fit_transform(data[['plasma_CA19_9', 'creatinine', 'LYVE1','REG1B', 'TFF1', 'REG1A']])

In [103]:
data

,age,sex,diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,6,0,0,0.000377,0.437326,0.037383,0.037715,0.049030,0.095606
1,54,0,0,NaN,0.225627,0.085284,0.067288,0.015698,0.017304
2,24,1,0,0.000226,0.178273,0.006089,0.072915,0.034557,NaN
3,34,1,0,0.000258,0.158774,0.000112,0.043150,0.010712,NaN
4,35,1,0,0.000290,0.038997,0.000031,0.046684,0.003079,NaN
...,...,...,...,...,...,...,...,...,...
585,41,1,2,NaN,0.114206,0.295438,0.111290,0.039356,NaN
586,44,0,2,NaN,0.197772,0.349142,0.012048,0.018430,NaN
587,36,1,2,NaN,0.323120,0.321244,0.206354,0.040263,NaN
588,48,0,2,NaN,0.314763,0.343515,0.146684,0.054144,NaN


In [104]:
data.corr()["diagnosis"]

age              0.308279
sex              0.169081
diagnosis        1.000000
plasma_CA19_9    0.263950
creatinine       0.074888
LYVE1            0.540384
REG1B            0.383516
TFF1             0.392613
REG1A            0.260110
Name: diagnosis, dtype: float64

In [105]:
X = data.drop('diagnosis', axis=1)
y = data['diagnosis']

In [106]:
y.value_counts()

1    208
2    199
0    183
Name: diagnosis, dtype: int64

In [107]:
acc_score = []
seeds = range(0,101)

for seed in seeds:
    print(seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    xgb = XGBClassifier(n_estimators=100,enable_categorical=True)
    xgb.fit(X_train, y_train)
    preds = xgb.predict(X_test)
    acc_xgb = (preds == y_test).sum().astype(float) / len(preds)*100
    acc_score.append(acc_xgb)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=acc_score.index(max(acc_score)))
xgb = XGBClassifier(n_estimators=100,enable_categorical=True)
xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)
acc_xgb = (preds == y_test).sum().astype(float) / len(preds)*100

In [146]:
acc_xgb

80.22598870056498

In [147]:
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score

cm=confusion_matrix(preds,y_test)
print(cm)

[[46 12  0]
 [ 8 43  8]
 [ 1  6 53]]
